In [5]:
import moviepy
from moviepy.editor import concatenate_videoclips
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.audio.AudioClip import CompositeAudioClip

from app import change_ffmpeg_fps

def add_logo(input_clip, output_clip_name, logo_file_name, logo_duration, position=("right", "top")):
    logo = (moviepy.ImageClip(logo_file_name)
            .set_duration(logo_duration)
            .resize(height=50) # if you need to resize...
            .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
            .set_pos(position))

    output_clip = moviepy.CompositeVideoClip([input_clip, logo])
    output_clip.write_videofile(output_clip_name)
    return output_clip

def edit_remove_part(input_filename, start_s, end_s, FPS=5):
    """Remove a segment of an .mp4 by and change its FPS. Keeps the audio intact by audio_codec='aac' parameter"""
    # input_video = change_ffmpeg_fps(VideoFileClip(input_filename), fps=FPS) # Load video and change fps for faster processing
    input_filename_no_ext, ext = input_filename.split('.')
    input_video = VideoFileClip(input_filename)

    # Remove part video and return result
    video = input_video.cutout(start_s, end_s)

    # Write Video
    output_filename = input_filename_no_ext + f"_RM_{start_s}s_to_{end_s}.{ext}"
    video.write_videofile(output_filename, audio_codec='aac')

def edit_change_order(input_filename, start_s, end_s, insert_s, FPS=5):
    """Take start_s to end_s out of the video and insert it at insert_s. Keeps the audio intact by audio_codec='aac' parameter"""
    # input_video = change_ffmpeg_fps(VideoFileClip(input_filename), fps=FPS) # Load video and change fps for faster processing
    input_filename_no_ext, ext = input_filename.split('.')
    input_video = VideoFileClip(input_filename)
    assert not (insert_s > start_s and insert_s < end_s), "Insert time can't be in cutout time!"
    assert (end_s > start_s), "End_s should be higher than start_s!"

    # Cut out a part and insert it somewhere else
    if insert_s < start_s and insert_s < end_s:
        part_start = input_video.subclip(0.0, insert_s)
        part_cutout = input_video.subclip(start_s, end_s)
        part_mid = input_video.subclip(insert_s, start_s)
        part_end = input_video.subclip(end_s, input_video.duration)
        video = concatenate_videoclips([part_start, part_cutout, part_mid, part_end])
    elif insert_s > start_s and insert_s > end_s:
        part_start = input_video.subclip(0.0, start_s)
        part_cutout = input_video.subclip(start_s, end_s)
        part_mid = input_video.subclip(end_s, insert_s)
        part_end = input_video.subclip(insert_s, input_video.duration)
        print(f"Part Start = {part_start.duration}, Part Cutout = {part_cutout.duration}, Part Mid = {part_mid.duration}, Part End = {part_end.duration}")
        video = concatenate_videoclips([part_start, part_mid, part_cutout, part_end])

    # Write Video
    output_filename = input_filename_no_ext + f"_CO_{start_s}s_to_{end_s}_at_{insert_s}.{ext}"
    video.write_videofile(output_filename, audio_codec='aac')

# edit_remove_part("videos/Ploumen.mp4", start_s = 5.0, end_s = 10.0)
# edit_change_order("videos/Ploumen.mp4", start_s = 5.0, end_s = 10.0, insert_s = 15.0)




DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.ipify.org:443
DEBUG:urllib3.connectionpool:https://api.ipify.org:443 "GET / HTTP/1.1" 200 15
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-initiated-analytics/ HTTP/1.1" 200 31
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-initiated-analytics/ HTTP/1.1" 200 31
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "GET /pkg-version HTTP/1.1" 200 20
DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.ipify.org:443


Using cache from '/Users/ijanssen/videomatch/gradio_cached_examples/15' directory. If method or examples have changed since last caching, delete this folder to clear cache.


DEBUG:urllib3.connectionpool:https://api.ipify.org:443 "GET / HTTP/1.1" 200 15
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-initiated-analytics/ HTTP/1.1" 200 31
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-initiated-analytics/ HTTP/1.1" 200 31
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "GET /pkg-version HTTP/1.1" 200 20
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.ipify.org:443
DEBUG:urllib3.connectionpool:https://api.ipify.org:443 "GET / HTTP/1.1" 200 15
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-initiated-analytics/ HTTP/1.1" 200 31


Part Start = 5.0, Part Cutout = 5.0, Part Mid = 5.0, Part End = 135.03
Moviepy - Building video videos/Ploumen_CO_5.0s_to_10.0_at_15.0.mp4.
MoviePy - Writing audio in Ploumen_CO_5.0s_to_10.0_at_15.0TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/Ploumen_CO_5.0s_to_10.0_at_15.0.mp4



Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_5.0s_to_10.0_at_15.0.mp4


In [21]:

import numpy as np

MAX = 130

# Get some random start_s and end_s pairs where start_s is always lower than end_s
rand_start_s = np.random.randint(low=0, high=MAX, size=5)
rand_end_s = np.random.randint(low=0, high=MAX, size=5)
rand_pairs = zip(rand_start_s, rand_end_s)
rand_pairs = [(x,y) if (x < y) else (y, x) for x, y in rand_pairs]

def get_insert_s(start_s, end_s, max=MAX):
    """ Get a insert_s that is outside the start_s and end_s """
    random_choice = bool(np.random.randint(low=0, high=2, size=1)[0])
    if random_choice:
        return np.random.randint(low=0, high=start_s, size=1)[0]
    else:
        return np.random.randint(low=end_s, high=MAX, size=1)[0]

rand_insert_s = [get_insert_s(x, y) for x, y in rand_pairs]

print(rand_pairs)
print(rand_insert_s)



[(51, 92), (14, 98), (64, 85), (63, 90), (63, 96)]
[112, 0, 53, 96, 123]


In [22]:
for pair, insert_s in zip(rand_pairs, rand_insert_s):
    edit_change_order("videos/Ploumen.mp4", start_s = pair[0], end_s = pair[1], insert_s = insert_s)

Part Start = 51.0, Part Cutout = 41, Part Mid = 20, Part End = 38.03
Moviepy - Building video videos/Ploumen_CO_51s_to_92_at_112.mp4.
MoviePy - Writing audio in Ploumen_CO_51s_to_92_at_112TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/Ploumen_CO_51s_to_92_at_112.mp4



Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_51s_to_92_at_112.mp4
Moviepy - Building video videos/Ploumen_CO_14s_to_98_at_0.mp4.
MoviePy - Writing audio in Ploumen_CO_14s_to_98_at_0TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/Ploumen_CO_14s_to_98_at_0.mp4



Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_14s_to_98_at_0.mp4
Moviepy - Building video videos/Ploumen_CO_64s_to_85_at_53.mp4.
MoviePy - Writing audio in Ploumen_CO_64s_to_85_at_53TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_64s_to_85_at_53.mp4
Part Start = 63.0, Part Cutout = 27, Part Mid = 6, Part End = 54.03
Moviepy - Building video videos/Ploumen_CO_63s_to_90_at_96.mp4.
MoviePy - Writing audio in Ploumen_CO_63s_to_90_at_96TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/Ploumen_CO_63s_to_90_at_96.mp4



Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_63s_to_90_at_96.mp4
Part Start = 63.0, Part Cutout = 33, Part Mid = 27, Part End = 27.03
Moviepy - Building video videos/Ploumen_CO_63s_to_96_at_123.mp4.
MoviePy - Writing audio in Ploumen_CO_63s_to_96_at_123TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos/Ploumen_CO_63s_to_96_at_123.mp4



Moviepy - Done !
Moviepy - video ready videos/Ploumen_CO_63s_to_96_at_123.mp4
